In [1]:
import pandas as pd
from pytz import country_timezones
import time
import numpy as np
import timezonefinder
from datetime import datetime as dt
import pytz
import calendar

In [2]:
data = pd.read_csv("../csv/exported_queries/mtime.csv") #import dataset
data.head() #check

,mtime,group_country,group_state,long,lat
0,1554151594772,us,OH,-81.57,41.08
1,1554268907000,us,MA,-71.10,42.37
2,1554147703000,de,NONE,11.58,48.14
3,1554157339000,us,CA,-118.00,34.14
4,1554220406070,us,FL,-80.06,26.71


In [3]:
data.group_state = data.group_state.str.lower() #lower case for group_state
data.head() #check

,mtime,group_country,group_state,long,lat
0,1554151594772,us,oh,-81.57,41.08
1,1554268907000,us,ma,-71.10,42.37
2,1554147703000,de,none,11.58,48.14
3,1554157339000,us,ca,-118.00,34.14
4,1554220406070,us,fl,-80.06,26.71


In [4]:
tf = timezonefinder.TimezoneFinder() #defining time zones on lat, long data
data['timezone_str'] = "nan"
count = 0
start = time.time()
for line in data.itertuples():
    count+=1
    try:
        data.timezone_str.at[line.Index] = tf.timezone_at(lat = data.iloc[line.Index]['lat'],
                                                          lng = data.iloc[line.Index]['long'])
    except Exception as e:
        print(e)
        break
    #if count == 10:
        #break
end = time.time()
print("task completed in "+str((end-start)/60)+" minutes")

task completed in 29.985913876692454 minutes


In [5]:
data.head() #check

,mtime,group_country,group_state,long,lat,timezone_str
0,1554151594772,us,oh,-81.57,41.08,America/New_York
1,1554268907000,us,ma,-71.10,42.37,America/New_York
2,1554147703000,de,none,11.58,48.14,Europe/Berlin
3,1554157339000,us,ca,-118.00,34.14,America/Los_Angeles
4,1554220406070,us,fl,-80.06,26.71,America/New_York


In [6]:
data.any().isnull() #check

mtime            False
group_country    False
group_state      False
long             False
lat              False
timezone_str     False
dtype: bool

In [7]:
data1 = data[data.lat != 0.0] #cleaned dataset
data1[data1.lat == 0] #check

,mtime,group_country,group_state,long,lat,timezone_str


In [8]:
data1.any().isnull() #another check, no more missing values

mtime            False
group_country    False
group_state      False
long             False
lat              False
timezone_str     False
dtype: bool

In [9]:
def rounder(t): #this function round the time to nearest hour
    if t.minute >= 30:
        if t.hour == 23:
            return t.replace(second=0, microsecond=0, minute=0, hour=0)
        else:
            return t.replace(second=0, microsecond=0, minute=0, hour= t.hour+1)
    else:
        return t.replace(second=0, microsecond=0, minute=0)

In [10]:
#data1['datetime_timezoned'] = "nan" #this method create problems with "SettingWithCopyWarning" don't know why
data1 = data1.assign(datetime_timezoned = "") #defining new column converting timestamp into datetime conditionally to time zones
start = time.time()
count = 0
for line in data1.itertuples():
    count+=1
    try: #this first line is to eventually create a column with the original UTC datetime format
        #data1.loc[line.Index, 'datetime'] = dt.fromtimestamp(timestamp = (data1.loc[line.Index, 'mtime'])/1000)
        data1.datetime_timezoned.at[line.Index] = rounder((dt.fromtimestamp
                                                   (timestamp = (data1.loc[line.Index, 'mtime'])/1000,
                                                    tz = pytz.timezone(data1.loc[line.Index, 'timezone_str'])
                                                   ))).strftime("%d/%m/%Y %H:%M:%S")
        #data1.datetime_timezoned.at[line.Index] = data1.iloc[line.Index]['datetime_timezoned'].strftime("%d/%m/%Y %H:%M:%S")
        #these two lines create first of all a column with the datetime timezoned based on the time zone of the group that
        #host the event, and in second time they change the format of that datetime
    except Exception as e:
        print(e)
        break
    #if count == 100:
        #break
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')

task completed in 5.602936724821727 minutes


In [11]:
data1.head()

,mtime,group_country,group_state,long,lat,timezone_str,datetime_timezoned
0,1554151594772,us,oh,-81.57,41.08,America/New_York,01/04/2019 17:00:00
1,1554268907000,us,ma,-71.10,42.37,America/New_York,03/04/2019 01:00:00
2,1554147703000,de,none,11.58,48.14,Europe/Berlin,01/04/2019 22:00:00
3,1554157339000,us,ca,-118.00,34.14,America/Los_Angeles,01/04/2019 15:00:00
4,1554220406070,us,fl,-80.06,26.71,America/New_York,02/04/2019 12:00:00


In [ ]:
#this code use a list to create the datetime_timezoned column, more efficient, 1 minute less than the other method
'''
datetime_timezoned = []
start = time.time()
count = 0
for line in data1.itertuples():
    count+=1
    try: #this first line is to eventually create a column with the original UTC datetime format
        #data1.loc[line.Index, 'datetime'] = dt.fromtimestamp(timestamp = (data1.loc[line.Index, 'mtime'])/1000)
        datetime_timezoned.append((dt.fromtimestamp(timestamp = (data1.loc[line.Index, 'mtime'])/1000,
                                                    tz = pytz.timezone(data1.loc[line.Index, 'timezone_str'])
                                                   )).strftime("%d/%m/%Y %H:%M:%S"))
        #data1.datetime_timezoned.at[line.Index] = data1.iloc[line.Index]['datetime_timezoned'].strftime("%d/%m/%Y %H:%M:%S")
        #these two lines create first of all a column with the datetime timezoned based on the time zone of the group that
        #host the event, and in second time they change the format of that datetime
    except Exception as e:
        print(e)
        break
    #if count == 10000:
        #break
data1['datetime_timezoned'] = datetime_timezoned
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')
'''

In [12]:
new = data1['datetime_timezoned'].str.split(" ", n = 1, expand = True) #create distinct columns date and time
data1['date'] = [calendar.day_name[dt.strptime(date, '%d/%m/%Y').weekday()] for date in new[0]] #transform date into weekdays
data1['time'] = new[1] #take time

In [13]:
data1.head() #check

,mtime,group_country,group_state,long,lat,timezone_str,datetime_timezoned,date,time
0,1554151594772,us,oh,-81.57,41.08,America/New_York,01/04/2019 17:00:00,Monday,17:00:00
1,1554268907000,us,ma,-71.10,42.37,America/New_York,03/04/2019 01:00:00,Wednesday,01:00:00
2,1554147703000,de,none,11.58,48.14,Europe/Berlin,01/04/2019 22:00:00,Monday,22:00:00
3,1554157339000,us,ca,-118.00,34.14,America/Los_Angeles,01/04/2019 15:00:00,Monday,15:00:00
4,1554220406070,us,fl,-80.06,26.71,America/New_York,02/04/2019 12:00:00,Tuesday,12:00:00


In [14]:
#writing the new csv
#data1.to_csv("../csv/exported_queries/mtime_manipulated.csv", index = False)